In [302]:
import requests
import concurrent.futures
from bs4 import BeautifulSoup

In [130]:
url = "https://www.investopedia.com/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
}

In [131]:
response = requests.get(url, headers=headers)

In [132]:
soup = BeautifulSoup(response.text, "html.parser")

In [143]:
titles = []

In [144]:
html_content = []

In [145]:
md_content = []

In [146]:
clean_content = []

In [294]:
top_links = []
def get_nav_links():
    """
    Gets all the links from the navbar.
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    nav_links = soup.find_all("li", class_="header-nav__list-item")
    nav_top_links = []
    for link in nav_links:
        if "simulator" not in link.a["href"] and "academy" not in link.a["href"]:
            nav_top_links.append(link.a["href"])
            top_links.append(link.a["href"])
    for link in nav_top_links:
        response = requests.get(link, headers=headers)
        soup_sublink = BeautifulSoup(response.text, "html.parser")
        sub_links = soup_sublink.find_all("li", class_="header-nav__sublist-item")
        for sub_link in sub_links:
            if "simulator" not in sub_link.a["href"] and "academy" not in sub_link.a["href"]:
                top_links.append(sub_link.a["href"])
    

In [296]:
get_nav_links()

In [299]:
morein_links = []
def get_morein_links(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    morein_links_bottom = soup.find_all("a", class_="mntl-taxonomysc-sibling-node mntl-text-link")
    for morein_link in morein_links_bottom:
        morein_links.append(morein_link["href"])

In [304]:
num_workers = 60

In [305]:
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    executor.map(get_morein_links, top_links)

In [308]:
top_links.extend(morein_links)

In [311]:
footer_links = []
def get_footer_links():
    footer_links_btm = soup.find_all("li", class_="comp terms-bar__item mntl-block")
    for link in footer_links_btm:
        footer_links.append(link.a["href"])

In [317]:
get_footer_links()

In [320]:
def get_faq_links(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    faq_links = soup.find_all("a", class_="accordion-content__feature-link mntl-text-link")
    return list(set([link["href"] for link in faq_links]))

In [321]:
def get_key_term_links(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    key_term_links = soup.find_all("a", class_="carousel-card__link mntl-text-link")
    return list(set([link["href"] for link in key_term_links]))

In [328]:
article_links = []
def get_article_links(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all('a', class_="comp mntl-card-list-items mntl-document-card mntl-card card card--no-image")
    faq_links = get_faq_links(url)
    key_term_links = get_key_term_links(url)
    article_links.extend(key_term_links + faq_links + list(set([link["href"] for link in links])))

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    executor.map(get_article_links, top_links)

In [ ]:
dataset_pd = {
    "url": articles,
    "title": 
    "html_content":
    "md_content":
    "clean_content":
}

In [40]:
def get_title_from_url(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.find("h1", class_="article-heading")
    return title.text.strip()

In [42]:
for link in articles:
    try:
        titles.append(get_title_from_url(link))
    except AttributeError:
        print(link) 

https://www.investopedia.com/markets-4689752
https://www.investopedia.com/portfolio-management-4689745
https://www.investopedia.com/finding-a-financial-advisor-6834423
https://www.investopedia.com/sustainable-investing-4427774
https://www.investopedia.com/alternative-investments-4427781
https://www.investopedia.com/investing-in-the-uk-7548232
https://www.investopedia.com/laws-and-regulations-4427769
https://www.investopedia.com/mutual-funds-4427787
https://www.investopedia.com/quantitative-analysis-5272131


MissingSchema: Invalid URL '#': No scheme supplied. Perhaps you meant https://#?